# Binary Answer Type

This notebook demonstrates how to generate forecasting questions with binary (yes/no) answers. Binary questions are the simplest and most common type, perfect for event prediction, outcome forecasting, and decision-making scenarios.

In [ ]:
%pip install lightningrod-ai

from IPython.display import clear_output
clear_output()

In [ ]:
import os
from datetime import datetime, timedelta
from lightningrod import (
    LightningRod,
    NewsSeedGenerator,
    QuestionGenerator,
    QuestionPipeline,
    WebSearchLabeler,
    FilterCriteria,
    AnswerType,
    AnswerTypeEnum,
)
from lightningrod._generated.models import QuestionRenderer

api_key = os.getenv("LIGHTNINGROD_API_KEY", "your-api-key-here")

client = LightningRod(api_key=api_key)

## Binary Answer Type Overview

Binary questions have two possible answers: Yes/No or True/False. They're ideal for:
- Event prediction ("Will X happen?")
- Outcome forecasting ("Will Y succeed?")
- Decision-making scenarios ("Should we do Z?")

Binary questions work with any data source: Google News, GDELT, or custom documents.

In [ ]:
answer_type = AnswerType(answer_type=AnswerTypeEnum.BINARY)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    interval_duration_days=7,
    search_query="technology product launches",
)

In [ ]:
question_generator = QuestionGenerator(
    instructions=(
        "Generate binary forecasting questions about technology product launches. "
        "Questions should be answerable with Yes or No."
    ),
    examples=[
        "Will Apple release a new iPhone this year?",
        "Will Google launch a new AI product in Q2?",
        "Will Microsoft release Windows 12 in 2025?",
    ],
    bad_examples=[
        "What features will the new iPhone have?",
        "When will Google launch the product?",
        "How much will Windows 12 cost?",
    ],
    filter_=FilterCriteria(
        rubric="The question must be answerable with Yes or No",
        min_score=0.7
    ),
    answer_type=answer_type,
)

In [ ]:
labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

renderer = QuestionRenderer(answer_type=answer_type)

## Run the Pipeline

Generate binary questions from the news articles.

In [ ]:
pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

dataset = client.transforms.run(pipeline_config, max_questions=30)

## Interpreting Binary Labels

Binary labels are boolean values (Yes/No, True/False) with confidence scores. Higher confidence indicates more certainty in the answer.

In [ ]:
samples = dataset.to_samples()

print(f"Generated {len(samples)} binary questions\n")

high_confidence = [s for s in samples if s.label and s.label.label_confidence > 0.8]
print(f"High confidence answers (>0.8): {len(high_confidence)}")
print(f"Total answers: {len([s for s in samples if s.label])}\n")

for i, sample in enumerate(samples[:5]):
    print(f"Question {i+1}: {sample.question.question_text if sample.question else 'N/A'}")
    if sample.label:
        print(f"  Answer: {sample.label.label}")
        print(f"  Confidence: {sample.label.label_confidence:.2f}")
        if sample.label.label_confidence > 0.8:
            print("  ✓ High confidence")
        elif sample.label.label_confidence > 0.5:
            print("  ~ Medium confidence")
        else:
            print("  ? Low confidence")
    print()

## When to Use Binary vs Other Answer Types

**Use Binary when:**
- You need simple yes/no answers
- Predicting events or outcomes
- Building decision-making systems
- You want the simplest evaluation

**Consider other types when:**
- You need numeric values → Use Continuous
- You need multiple options → Use Multiple Choice
- You need open-ended explanations → Use Free Response